<a href="https://colab.research.google.com/github/jogfx/ADL_sbert/blob/main/SBERT_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.0 MB/s eta 0:00:00


In [2]:
!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account a

In [3]:
import pandas as pd
import gradio as gr
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# Load SBERT model
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# Load the data from the provided URL
url = "https://raw.githubusercontent.com/jogfx/ADL_sbert/main/job_skills.csv"
data = pd.read_csv(url)

In [6]:
# Display the first few rows of the DataFrame
print(data.head())

  Company                                              Title  \
0  Google                       Google Cloud Program Manager   
1  Google  Supplier Development Engineer (SDE), Cable/Con...   
2  Google  Data Analyst, Product and Tools Operations, Go...   
3  Google            Developer Advocate, Partner Engineering   
4  Google     Program Manager, Audio Visual (AV) Deployments   

                       Category                          Location  \
0            Program Management                         Singapore   
1  Manufacturing & Supply Chain                   Shanghai, China   
2           Technical Solutions       New York, NY, United States   
3           Developer Relations  Mountain View, CA, United States   
4            Program Management      Sunnyvale, CA, United States   

                                    Responsibilities  \
0  Shape, shepherd, ship, and show technical prog...   
1  Drive cross-functional activities in the suppl...   
2  Collect and analyze data to d

In [7]:
# Function to clean text
def clean_text(text):
    # Remove special characters and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert text to lowercase
    text = text.lower()
    return text

In [8]:
# Function to tokenize text
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

In [9]:
# Initialize stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [10]:
# Function to remove stopwords and lemmatize tokens
def preprocess_tokens(tokens):
    # Remove stopwords and lemmatize tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return tokens

In [11]:
# Function to preprocess job descriptions
def preprocess_description(row):
    # Convert float values to empty strings
    responsibilities = str(row['Responsibilities'])
    min_qualifications = str(row['Minimum Qualifications'])
    preferred_qualifications = str(row['Preferred Qualifications'])

    # Concatenate the descriptions
    description = ' '.join([responsibilities, min_qualifications, preferred_qualifications])

    # Clean and preprocess the concatenated description
    cleaned_description = clean_text(description)
    tokens = tokenize_text(cleaned_description)
    preprocessed_tokens = preprocess_tokens(tokens)
    return ' '.join(preprocessed_tokens)

In [12]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [13]:
# Apply preprocessing to create a new 'Description' column
data['Description'] = data.apply(preprocess_description, axis=1)

In [14]:
# Encode job descriptions using SBERT
description_embeddings = sbert_model.encode(data['Description'])

In [15]:
# Function to perform semantic search
def semantic_search(query_embedding, job_description_embeddings, top_k=5):
    # Compute cosine similarity between query embedding and job description embeddings
    similarities = cosine_similarity(query_embedding.reshape(1, -1), job_description_embeddings)[0]
    # Get indices of top-k most similar job descriptions
    top_indices = similarities.argsort()[-top_k:][::-1]
    return top_indices

In [16]:
# Example user query or resume
user_query = "software engineer with experience in machine learning"

In [17]:
# Encode user query using SBERT
user_query_embedding = sbert_model.encode([user_query])[0]

In [18]:
# Perform semantic search
top_job_indices = semantic_search(user_query_embedding, description_embeddings)

In [19]:
# Get top-k most relevant job postings
top_job_postings = data.iloc[top_job_indices]

In [20]:
# Display the top-k most relevant job postings
print(top_job_postings[['Title', 'Description']])

                                                Title  \
127                Silicon Design Monitoring Engineer   
156                      Software Engineering Manager   
52   Technical Program Manager, Corporate Engineering   
255             Software Engineering Manager, G Suite   
65            Software Engineer, Performance Modeling   

                                           Description  
127  design deploy storage compute system reporting...  
156  manage team software engineer including task p...  
52   develop track program plan consult solution te...  
255  manage team software engineer including task p...  
65   develop simulation model different abstraction...  


In [21]:
def recommend_jobs(query):
    query_embedding = sbert_model.encode([query])[0]
    top_job_indices = semantic_search(query_embedding, description_embeddings)
    top_job_postings = data.iloc[top_job_indices][['Title', 'Description']]

    # Format job postings as HTML table
    html_output = "<table><tr><th>Title</th><th>Description</th></tr>"
    for index, row in top_job_postings.iterrows():
        title = row['Title']
        description = row['Description']
        html_output += f"<tr><td>{title}</td><td>{description}</td></tr>"
    html_output += "</table>"

    return html_output

In [25]:
gr.Interface(recommend_jobs,
              inputs="text",
              outputs="html",
              title="Data/Technical Job Recommendation System",
              description="Enter your highest level of completed education to get relevant job postings.").launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2a5bf8a567c3944e6e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
